# 🎨 GenImg x402 v2 Payment Demo

This notebook demonstrates how to use the **x402 v2 payment protocol** to pay for AI-generated images with USDC on Optimism.

## What is x402?

[x402](https://github.com/coinbase/x402) is an open protocol by Coinbase that enables HTTP-native payments. When a server returns a `402 Payment Required` response, the client automatically:

1. Parses the payment requirements from the response headers
2. Creates a cryptographic payment authorization (EIP-3009)
3. Retries the request with the payment attached
4. Receives the paid content

**No approve transactions needed!** x402 uses EIP-3009 `transferWithAuthorization` for gasless, one-step payments.

## 🔧 Prerequisites

Before running this notebook, you need:

1. **Deno Jupyter Kernel** - Install with:
   ```bash
   deno jupyter --install
   ```

2. **A `.env` file** in the `notebooks/` directory with:
   ```env
   TEST_WALLET_PRIVATE_KEY=your_private_key_without_0x
   ```

3. **USDC on Optimism** - Either:
   - **Testnet (Sepolia)**: Get test USDC from a faucet
   - **Mainnet**: Real USDC (~$0.001 per image)

4. **Running GenImg Server** - Start locally:
   ```bash
   cd scw_js
   NODE_ENV=test node ./genimg_x402_token.js
   ```

## 📚 How to Use This Notebook

1. **Cell 1**: Run to import dependencies
2. **Cell 2**: Configure network (testnet/mainnet) and prompt
3. **Cell 3**: Setup x402 client
4. **Cell 4**: Execute payment and generate image
5. **Cell 5**: Display the generated image

Run the cells in order from top to bottom.

## 📦 Step 1: Import Dependencies

First, we import the x402 client libraries and viem for wallet management. 

**Packages used:**
- `@x402/fetch` - HTTP client with automatic payment handling
- `@x402/evm` - EVM-specific payment scheme (EIP-3009)
- `viem` - Ethereum library for wallet and chain configuration

In [20]:
// 📦 Import x402 Buyer Dependencies
// Following official Quickstart Guide exactly!

import { x402Client, wrapFetchWithPayment, x402HTTPClient } from "npm:@x402/fetch@^2.0.0";
import { registerExactEvmScheme } from "npm:@x402/evm@^2.0.0/exact/client";

// Viem for wallet and chains
import { privateKeyToAccount } from "npm:viem@^2.0.0/accounts";
import { optimism, optimismSepolia } from "npm:viem@^2.0.0/chains";

// Load environment variables
import { load } from "https://deno.land/std@0.224.0/dotenv/mod.ts";
const env = await load({ export: true });

console.log("✅ Dependencies loaded!");
console.log("   x402Client:", typeof x402Client);
console.log("   wrapFetchWithPayment:", typeof wrapFetchWithPayment);
console.log("   registerExactEvmScheme:", typeof registerExactEvmScheme);
console.log("   Chains:", { optimism: optimism.name, optimismSepolia: optimismSepolia.name });

✅ Dependencies loaded!
   x402Client: function
   wrapFetchWithPayment: function
   registerExactEvmScheme: function
   Chains: { optimism: "OP Mainnet", optimismSepolia: "OP Sepolia" }


## ⚙️ Step 2: Configuration

Configure your wallet, network, and prompt below.

**Environment Variables** (loaded from `.env`):
- `TEST_WALLET_PRIVATE_KEY` - Your wallet private key (without `0x` prefix)

**Network Options:**
| Network | Chain ID | USDC | Use Case |
|---------|----------|------|----------|
| Optimism Sepolia | `eip155:11155420` | Test USDC | Development/Testing |
| Optimism Mainnet | `eip155:10` | Real USDC | Production |

⚠️ **Warning**: Setting `USE_MAINNET = true` uses **real money**!

In [ ]:
// ⚙️ Configuration - lädt aus .env Datei
// const PRIVATE_KEY = env.TEST_WALLET_PRIVATE_KEY;
const PRIVATE_KEY = env.NFT_WALLET_PRIVATE_KEY;
const SERVICE_URL = "http://localhost:8082";
const FACILITATOR_URL = "https://facilitator.fretchen.eu";
const PROMPT = "A cute robot painting a picture";

// ⚠️ NETWORK SELECTION - Change this to switch between testnet and mainnet
const USE_MAINNET = false;  // Set to true for Optimism Mainnet with REAL MONEY

// Network configuration based on selection
const NETWORK = USE_MAINNET ? {
  // Optimism Mainnet - REAL MONEY! 💰
  chain: optimism,
  chainId: "eip155:10",
  name: "Optimism Mainnet",
  usdc: "0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85",
  decimals: 6,
} : {
  // Optimism Sepolia - Testnet 🧪
  chain: optimismSepolia,
  chainId: "eip155:11155420",
  name: "Optimism Sepolia",
  usdc: "0x5fd84259d66Cd46123540766Be93DFE6D43130D7",
  decimals: 6,
};

// Show configuration
console.log("📋 Configuration:");
console.log(`   PRIVATE_KEY: ${PRIVATE_KEY ? PRIVATE_KEY.slice(0, 6) + "..." + PRIVATE_KEY.slice(-4) : "❌ NOT FOUND"}`);
console.log(`   SERVICE_URL: ${SERVICE_URL}`);
console.log(`   FACILITATOR_URL: ${FACILITATOR_URL}`);

if (USE_MAINNET) {
  console.log(`\n🚨 WARNING: Using REAL MONEY on ${NETWORK.name}!`);
  console.log(`⚠️  Make sure this is intentional!`);
} else {
  console.log(`\n🧪 Using testnet: ${NETWORK.name}`);
}

console.log(`\n📍 Network: ${NETWORK.name} (${NETWORK.chainId})`);
console.log(`💰 USDC Contract: ${NETWORK.usdc}`);
console.log(`🎨 Prompt: "${PROMPT}"`);

📋 Configuration:
   PRIVATE_KEY: f9c9c4...aa47
   SERVICE_URL: http://localhost:8082
   FACILITATOR_URL: https://facilitator.fretchen.eu

🚨 WARNING: Using REAL MONEY on Optimism Mainnet!
⚠️  Make sure this is intentional!

📍 Network: Optimism Mainnet (eip155:10)
💰 USDC Contract: 0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85
🎨 Prompt: "A cute robot painting a picture"


## 🔑 Step 3: Setup x402 Client

Create the x402 client and register the EVM payment scheme. This is the same pattern used in the [official x402 Quickstart](https://github.com/coinbase/x402).

The `wrapFetchWithPayment` function wraps the standard `fetch` API to automatically handle 402 responses.

In [22]:
// 🔧 Setup x402 client - Exactly like Quickstart!

console.log("🔧 Setting up x402 client...");

// Create signer (wallet)
const signer = privateKeyToAccount(`0x${PRIVATE_KEY}`);

// Create x402 client
const client = new x402Client();

// Register EVM scheme (tells client how to create EIP-3009 signatures)
registerExactEvmScheme(client, { signer });

// Wrap fetch with payment handling
const fetchWithPayment = wrapFetchWithPayment(fetch, client);

console.log("✅ x402 client ready");
console.log("   Will use server's first offered network");
console.log("   Server controls via sepoliaTest flag");

🔧 Setting up x402 client...
✅ x402 client ready
   Will use server's first offered network
   Server controls via sepoliaTest flag


## 🚀 Step 4: Generate Image with Payment

This is the main action! The `fetchWithPayment` function:

1. Sends initial POST request to the GenImg service
2. Receives `402 Payment Required` with USDC payment requirements
3. Creates EIP-3009 signature for the payment amount
4. Retries request with payment authorization header
5. Returns the generated image URL and NFT token ID

**What you get back:**
- `imageUrl` - URL to the AI-generated image
- `tokenId` - Your NFT certificate on Optimism
- Payment receipt in response headers

In [23]:
// 🚀 Make paid request
// (x402HTTPClient already imported in Cell 1)

console.log("🚀 Making paid request...");

// Make request - payment is handled automatically (exactly like Quickstart!)
const response = await fetchWithPayment(`${SERVICE_URL}`, {
  method: "POST",
  headers: {
    "Content-Type": "application/json",
  },
  body: JSON.stringify({ 
    prompt: PROMPT,
    sepoliaTest: !USE_MAINNET  // Server will restrict to Sepolia if true
  }),
});

console.log(`📡 Response Status: ${response.status}`);

if (response.ok) {
  const result = await response.json();
  console.log("\n🎉 SUCCESS!");
  console.log(`🖼️  Image URL: ${result.imageUrl}`);
  console.log(`🎫 NFT Token ID: ${result.tokenId}`);
  console.log(`📍 Contract: ${result.contractAddress}`);
  
  // Get payment receipt from response headers (exactly like Quickstart!)
  const httpClient = new x402HTTPClient(client);
  const paymentResponse = httpClient.getPaymentSettleResponse(
    (name) => response.headers.get(name)
  );
  
  if (paymentResponse) {
    console.log("\n✅ Payment settled:");
    console.log(`   Transaction: ${paymentResponse.transaction}`);
    console.log(`   Network: ${paymentResponse.network}`);
  }
  
  console.log("\n💡 fetchWithPayment automatically:");
  console.log("   1. Received 402 Payment Required");
  console.log("   2. Parsed payment requirements");
  console.log("   3. Created EIP-3009 signature");
  console.log("   4. Retried with payment");
  console.log("   5. Got successful response!");
} else {
  const error = await response.text();
  console.error(`❌ Error: ${error}`);
  throw new Error(`Request failed: ${response.status}`);
}

🚀 Making paid request...
📡 Response Status: 200

🎉 SUCCESS!
🖼️  Image URL: undefined
🎫 NFT Token ID: 170
📍 Contract: undefined

✅ Payment settled:
   Transaction: undefined
   Network: eip155:10

💡 fetchWithPayment automatically:
   1. Received 402 Payment Required
   2. Parsed payment requirements
   3. Created EIP-3009 signature
   4. Retried with payment
   5. Got successful response!


## 🎨 Step 5: Display Generated Image

After a successful payment, display the generated image. Copy the `imageUrl` from the output above into the cell below.

In [25]:
// Zeige das generierte Bild an
const imageUrl = "https://my-imagestore.s3.nl-ams.scw.cloud/images/image_1766753789601_9cdf583207d1.jpg";

if (imageUrl) {
  // Deno Jupyter API für HTML-Anzeige
  await Deno.jupyter.display({
    "text/html": `
      <div style="text-align: center; padding: 20px;">
        <h3>🎨 Generated Image</h3>
        <img src="${imageUrl}" alt="Generated Image" style="max-width: 100%; border-radius: 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);" />
        <p style="color: #666; margin-top: 10px; font-size: 12px;">URL: ${imageUrl}</p>
      </div>
    `
  }, { raw: true });
}

🎨 Generated Image 
 
 URL: https://my-imagestore.s3.nl-ams.scw.cloud/images/image_1766753789601_9cdf583207d1.jpg

---

## 📝 Troubleshooting

### Common Errors

| Error | Cause | Solution |
|-------|-------|----------|
| `Network not supported` | Facilitator doesn't support the chain | Check `/supported` endpoint |
| `Insufficient USDC balance` | Not enough USDC in wallet | Get testnet USDC or buy on mainnet |
| `Payment verification failed` | Invalid signature or expired | Retry the request |
| `Service configuration error` | Server wallet has no ETH for gas | Fund the server wallet |

### Useful Links

- [x402 Protocol Documentation](https://github.com/coinbase/x402)
- [Optimism Faucet](https://faucet.optimism.io/)
- [USDC Contract Addresses](https://developers.circle.com/stablecoins/docs/usdc-on-test-networks)

### Architecture

```
┌─────────┐     1. POST /genimg      ┌──────────┐
│  Client │ ──────────────────────▶  │  Server  │
│ (this   │                          │ (genimg) │
│ notebook)│  ◀────────────────────  │          │
└─────────┘   2. 402 + X-Payment     └──────────┘
     │                                     │
     │  3. Create EIP-3009 signature       │
     ▼                                     │
┌─────────┐   4. POST + payment      ┌─────────────┐
│  x402   │ ───────────────────────▶ │ Facilitator │
│ Client  │                          │ (verifies)  │
└─────────┘                          └─────────────┘
     │                                     │
     │  5. Success + NFT tokenId           │
     ◀─────────────────────────────────────┘
```